In [1]:
!git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...

Updating files:  15% (2191/13996)

Updating files:  16% (2240/13996)

Updating files:  17% (2380/13996)

Updating files:  18% (2520/13996)

Updating files:  19% (2660/13996)

Updating files:  20% (2800/13996)

Updating files:  21% (2940/13996)

Updating files:  22% (3080/13996)

Updating files:  23% (3220/13996)

Updating files:  24% (3360/13996)

Updating files:  25% (3499/13996)

Updating files:  26% (3639/13996)

Updating files:  27% (3779/13996)

Updating files:  28% (3919/13996)

Updating files:  29% (4059/13996)

Updating files:  29% (4109/13996)

Updating files:  30% (4199/13996)

Updating files:  31% (4339/13996)

Updating files:  32% (4479/13996)

Updating files:  33% (4619/13996)

Updating files:  34% (4759/13996)

Updating files:  35% (4899/13996)

Updating files:  36% (5039/13996)

Updating files:  37% (5179/13996)

Updating files:  38% (5319/13996)

Updating files:  39% (5459/13996)

Updating files:  40% (5599/13996)

Updating files:  

In [2]:
import csv
data = {}
with open('facial_expressions/data/legend.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        key = row[2].lower()
        if key in data:
            if key != 'facial-expressions_2868588k.jpg':
                data[key].append(row[1])
        else:
            if key != 'facial-expressions_2868588k.jpg':
                data[key] = [row[1]]

In [3]:
emotion_list = list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [5]:
import os

os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [6]:
for emotion in emotion_list:
    os.mkdir(os.path.join('master_data/training/', emotion))
    os.mkdir(os.path.join('master_data/testing/', emotion))

In [7]:
from shutil import copyfile
split_size = 0.8

for emotion, images in data.items():
    train_size = int(split_size*len(images))
    train_images = images[:train_size]
    test_images = images[train_size:]
    for image in train_images:
        if(image == 'facial-expressions_2868582k.jpg'):
            continue
        if(image == 'facial-expressions_2868584k.jpg'):
            continue
        if(image == 'facial-expressions_2868585k.jpg'):
            continue
        if(image == 'facial-expressions_2868588k.jpg'):
            continue
        source = os.path.join('facial_expressions/images', image)
        dest = os.path.join('master_data/training', emotion, image)
        copyfile(source, dest)
    for image in test_images:
        if(image == 'facial-expressions_2868582k.jpg'):
            continue
        if(image == 'facial-expressions_2868584k.jpg'):
            continue
        if(image == 'facial-expressions_2868585k.jpg'):
            continue
        if(image == 'facial-expressions_2868588k.jpg'):
            continue
        source = os.path.join('facial_expressions/images', image)
        dest = os.path.join('master_data/testing', emotion, image)
        copyfile(source, dest)

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [11]:
model = tf.keras.models.Sequential([
      Conv2D(16, (3,3), activation='relu', input_shape = (100, 100, 3)),
      MaxPooling2D(2, 2),
      Conv2D(32, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Conv2D(64, (3,3), activation='relu'),
      MaxPooling2D(2, 2),
      Flatten(),
      Dense(2048, activation='relu'),
      Dense(8, activation='softmax')
])
model.compile(optimizer = Adam(lr = 0.01), loss = 'categorical_crossentropy', metrics = ['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)             

In [12]:
train_dir = 'master_data/training'
test_dir = 'master_data/testing'

train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory(
                                                    train_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                  )

test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory(
                                                    test_dir,
                                                    target_size = (100, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 128
                                                  )

Found 10937 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [13]:
es = EarlyStopping(monitor='val_acc', patience = 2, min_delta=0.01)

In [14]:
model.fit_generator(train_generator,
                    epochs = 10,
                    verbose = 1,
                    validation_data = test_generator,
                    callbacks = [es])

Epoch 1/10
86/86 [==============================] - 46s 535ms/step - loss: 1.8050 - acc: 0.4646 - val_loss: 1.0411 - val_acc: 0.4384
Epoch 2/10
86/86 [==============================] - 12s 143ms/step - loss: 1.0416 - acc: 0.4977 - val_loss: 1.0315 - val_acc: 0.5518
Epoch 3/10
86/86 [==============================] - 12s 140ms/step - loss: 0.9998 - acc: 0.5577 - val_loss: 0.9908 - val_acc: 0.5835
Epoch 4/10
86/86 [==============================] - 12s 140ms/step - loss: 0.8424 - acc: 0.6902 - val_loss: 0.9391 - val_acc: 0.6568
Epoch 5/10
86/86 [==============================] - 12s 140ms/step - loss: 0.7599 - acc: 0.7344 - val_loss: 0.9983 - val_acc: 0.6389
Epoch 6/10
86/86 [==============================] - 12s 140ms/step - loss: 0.7476 - acc: 0.7377 - val_loss: 0.9734 - val_acc: 0.6287
